# <a style="color: red;">filter for mvm emails</a>

# <u>What makes someone engaged?</u>

### Determine engagement based on `donations` and `actions` 
### Rank that engagement using a <a style="color: red;">R</a><a style="color: orange;">A</a><a style="color: green;">G</a> system


### `Donations`
 - Frequency:   How often is this person donating?
 - Size:   How much is this person donating?


### `Actions`
 - Petitions
 - Surveys
 - Email engagement
 - Volunteering

## Donations

In [27]:
import pandas as pd 
import os  

In [28]:
donations = pd.read_csv("../data/raw/donations.csv")
report = pd.read_csv("../data/raw/main_report.csv")



Alot of people aren't going to donate, so we can make the <a style='color : red'>red</a> group, all the people who haven't donated<br>
Since <a style='color : red'>red</a> is anyone who hasn't donated, <a style='color : orange'>amber</a> can be anyone who *has* donated

In [29]:
unique_donators = donations.nunique()["Email"]
print(f"Unique donators: {unique_donators}")

Unique donators: 3739


In [30]:
# Group all donations by email, i.e. using email as a unique identifier
columns_to_group = ["Email", "Donation Amount"]
donations_by_email = donations[columns_to_group].groupby("Email").sum().reset_index()

# Finding average donation metrics
average_donation_size = donations_by_email["Donation Amount"].mean()
median_donation_size = donations_by_email["Donation Amount"].median()

count_of_each_email = donations.groupby("Email").size()
average_donation_count = count_of_each_email.mean()
median_donation_count = count_of_each_email.median()
max_donation_count = count_of_each_email.max()

quartiles = count_of_each_email.quantile([0.25, 0.5, 0.75])

print(f"Average donation size: £{average_donation_size:.2f}")
print(f"Median donation size: £{median_donation_size:.2f}")
print(f"Average donation count: {average_donation_count:.0f}")
print(f"Median donation count: {median_donation_count:.0f}")
print(f"Max donation count: {max_donation_count:.0f}")

print(quartiles)

Average donation size: £51.71
Median donation size: £33.00
Average donation count: 6
Median donation count: 4
Max donation count: 46
0.25     1.0
0.50     4.0
0.75    11.0
dtype: float64


In [31]:

'''
Make a new column in main report for donation count and total value 
Remove old total value as it is outside the time scope of this project/data
Populate donation_count and total_donation_value columns
'''

donations_by_email.rename(columns={'Email': 'email'}, inplace=True)

# email + total value is already part of donations_by_email
donation_subset_to_join = pd.DataFrame(donations_by_email)

# for each email, count number of donations made in donations 
donation_subset_to_join["donation_count"] = donation_subset_to_join["email"].map(donations["Email"].value_counts())



donation_subset_to_join.set_index("email", inplace=True)



report = report.merge(donation_subset_to_join, on="email", how="left")

# new_report.columns.to_list()


The <a style='color : green'>green</a> group is somewhat arbitrary, but after meeting with stakeholders we decided on the top **25%** of donators 

## Actions

To obtain 'actions', which include petitions, events, ticketed events, forms, surveys, letter campaigns and call campaigns I would need every instance of each of these, and count actions for every supporter. <br>
Due to the ActionNetwork subscription tier MVM has, I cannot access the data directly.<br>
ActionNetwork offers a service on its site which can be used to work around this, and due to the time constraints of this project, I will make full use of it.

This will require generating reports with no-code filtering solutions, and manually assessing the values to build an understanding of overall action data, which is not something MVM currently has

After meeting with stakeholders we decided the following margins for the <a style="color: red;">R</a><a style="color: orange;">A</a><a style="color: green;">G</a> system, covering data since 01/01/25

<a style="color: red;">Red:</a> No actions or their actions completed is in the lowest quartile<br/>
<a style="color: orange;">Amber:</a> Their actions completed is between the first and fourth quartile <br/>
<a style="color: green;">Green:</a> Their actions completed is in the top quartile 

I will also need to remove any MVM native emails as they often have a high number of actions which will skew data 

In [32]:

'''
Extracting the action data and putting it into the full report as a new column 
'''

file_path = "../data/raw/"
# add new column to `report`
# add value according to file 

report["actions_taken"] = float("nan")

for file_name in os.listdir(file_path):
    try:
        action_count = int(file_name.split("-")[0])
        df = pd.read_csv(file_path+file_name)
        df.dropna(inplace=True)
        report.loc[report['email'].isin(df["email"]), "actions_taken"] = action_count
    except Exception as e:
        print(f"im lazy: {e}")
# report.loc[report["actions_taken"] == 11, ["email", "actions_taken"]]




im lazy: invalid literal for int() with base 10: '.gitkeep'
im lazy: invalid literal for int() with base 10: 'action_data'
im lazy: invalid literal for int() with base 10: 'donations.csv'
im lazy: invalid literal for int() with base 10: 'main_report.csv'


In [33]:
'''
Finding statistics around actions taken 
'''
actions_without_0 = report.loc[report["actions_taken"] > 0, "actions_taken"]

average_action_count = actions_without_0.mean()
median_action_count = actions_without_0.median()

action_quartiles = actions_without_0.quantile([0.25, 0.5, 0.75])

print(f"Average actions taken {average_action_count} when excluding 0 actions taken")
print(f"Median actions taken {median_action_count} when excluding 0 actions taken")

print(action_quartiles)



Average actions taken nan when excluding 0 actions taken
Median actions taken nan when excluding 0 actions taken
0.25   NaN
0.50   NaN
0.75   NaN
Name: actions_taken, dtype: float64


## Evaluating Engagement 

In [34]:
'''
Add two new columns which are how far they are distributed for donation engagement and action in engagement 
Add a third column which combines the 2 somehow 
'''

# total value of a person, displayed as their percentile score from -100 to 100 and added to 'report'
donations_by_email["donation_size_score_100"] = (donations_by_email["Donation Amount"].rank(pct=True))

score_subset_to_join = donations_by_email[["email", "donation_size_score_100"]]

score_subset_to_join.set_index("email", inplace=True)

report = report.merge(score_subset_to_join, on="email", how="left")

report.columns

Index(['email', 'can2_user_tags', 'uuid', 'can2_lifetime_value',
       'Donation Amount', 'donation_count', 'actions_taken',
       'donation_size_score_100'],
      dtype='object')

In [35]:

donation_subset_to_join["donation_count"] = donation_subset_to_join["donation_count"].astype(float)
try:
    donations_by_email.set_index("email", inplace=True)
except:
    pass
donations_by_email["donation_freq_score_100"] = (donation_subset_to_join["donation_count"].rank(pct=True))


donations_by_email.loc[donations_by_email.index == "05.boulder.logon@icloud.com"]

,Donation Amount,donation_size_score_100,donation_freq_score_100
email,,,
05.boulder.logon@icloud.com,220.0,0.977802,0.780155


In [36]:
donations_ranks_to_join = donations_by_email.copy()
donations_ranks_to_join.drop("Donation Amount", axis=1, inplace=True)
donations_ranks_to_join.drop("donation_size_score_100", axis=1, inplace=True)
donations_ranks_to_join




,donation_freq_score_100
email,
00.samroberts.00@gmail.com,0.780155
05.boulder.logon@icloud.com,0.780155
123heatherv@gmail.com,0.607917
1407adrian@gmail.com,0.193635
1943gd@gmail.com,0.193635
...,...
zacyeo@hotmail.com,0.193635
zaidshopping@pm.me,0.193635
zakpreston1@gmail.com,0.780155


In [37]:
report = report.merge(donations_ranks_to_join, on="email", how="left")

In [38]:
donations_ranks_to_join.loc[donations_ranks_to_join["donation_freq_score_100"] == 1]

,donation_freq_score_100
email,
stewart.reddaway@gmail.com,1.0


In [39]:
report["actions_taken_score_100"] = (report["actions_taken"].rank(pct=True))

### What makes a supporter valuable?

Is someone who's in the 99th percentile of actions taken, more or less valuable than someone in the same percentile of donation value?


*i.e. how much money is an action worth?*

After meeting with stakeholders, we decided to equate the two values as at a high level, supporter actions are the goal of MVM

In [40]:
'''
combine donation size and actions taken score into 1 value
'''

report["engagement_score_100"] = (report["donation_size_score_100"].fillna(0) + report["actions_taken_score_100"].fillna(0)) / 2
# report["engagement_RAG"] = 

In [41]:
report

,email,can2_user_tags,uuid,can2_lifetime_value,Donation Amount,donation_count,actions_taken,donation_size_score_100,donation_freq_score_100,actions_taken_score_100,engagement_score_100
0,j.chapman48@btinternet.com,Temporary / 2023-05 / SortTheSystem Ask For Zoom,0be4e86e-ba50-11e3-a134-12313d1576ca,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
1,toniobg@hotmail.com,"Christmas Appeal Letter 2025, Donor, Donor / C...",11f160de-ba50-11e3-a134-12313d1576ca,292,102.3,11.0,NaN,0.874699,0.780155,NaN,0.43735
2,ritaashworth@yahoo.co.uk,"2025_Target_Seat, Has_LocalGroup_Any, Temporar...",2b85b072-ba50-11e3-a134-12313d1576ca,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
3,richardsonlynne@yahoo.co.uk,NaN,efed26e6-2793-44e9-ada6-f016abdecaf2,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
4,philip.m.green@gmail.com,"2025_Target_Seat, Has_LocalGroup_Any",156706b8-b107-431d-8d33-41c670511b56,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
...,...,...,...,...,...,...,...,...,...,...,...
110210,dakinjames14@yahoo.com,Admin / Ladder / Completed Welcome Ladder,fcad4eb9-131c-4458-ae1b-77a5c27b8b3a,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
110211,cwmorle37@gmail.com,Admin / Ladder / Completed Welcome Ladder,6cf1406b-4d08-48d2-983d-ea80cb98bde5,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
110212,stevepotelogs@btinternet.com,Admin / Ladder / Completed Welcome Ladder,4ab73ebd-e5ec-47d1-b817-2785cdaf3229,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
110213,mark.farrow13@sky.com,Admin / Ladder / Completed Welcome Ladder,91afc689-deb0-4aeb-a0b8-97b3995327c7,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
